In [1]:
from pathlib import Path
from src.configs import ModelArgs, DataArgs
from src.model import EvolutionOperator
from src.data import get_dataset
data_path = Path("/media/data/pnovelli/md_datasets/DESRES_folding_trajs/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein")
trajectory_files = [str(traj) for traj in data_path.glob("*.dcd")]
top = next(data_path.glob("*.pdb")).__str__()

Data loading

In [2]:
data_args = DataArgs(stride=100, cutoff=6.0, system_selection="all and not type H")

dataset = get_dataset(trajectory_files, top, DataArgs())

[TYR1-N, TYR1-H, TYR1-H2, TYR1-H3, TYR1-CA, TYR1-HA, TYR1-CB, TYR1-HB3, TYR1-HB2, TYR1-CG, TYR1-CD1, TYR1-HD1, TYR1-CE1, TYR1-HE1, TYR1-CZ, TYR1-OH, TYR1-HH, TYR1-CD2, TYR1-HD2, TYR1-CE2, TYR1-HE2, TYR1-C, TYR1-O, TYR2-N, TYR2-H, TYR2-CA, TYR2-HA, TYR2-CB, TYR2-HB3, TYR2-HB2, TYR2-CG, TYR2-CD1, TYR2-HD1, TYR2-CE1, TYR2-HE1, TYR2-CZ, TYR2-OH, TYR2-HH, TYR2-CD2, TYR2-HD2, TYR2-CE2, TYR2-HE2, TYR2-C, TYR2-O, ASP3-N, ASP3-H, ASP3-CA, ASP3-HA, ASP3-CB, ASP3-HB3, ASP3-HB2, ASP3-CG, ASP3-OD1, ASP3-OD2, ASP3-C, ASP3-O, PRO4-N, PRO4-CD, PRO4-HD3, PRO4-HD2, PRO4-CA, PRO4-HA, PRO4-CB, PRO4-HB3, PRO4-HB2, PRO4-CG, PRO4-HG3, PRO4-HG2, PRO4-C, PRO4-O, GLU5-N, GLU5-H, GLU5-CA, GLU5-HA, GLU5-CB, GLU5-HB3, GLU5-HB2, GLU5-CG, GLU5-HG3, GLU5-HG2, GLU5-CD, GLU5-OE1, GLU5-OE2, GLU5-C, GLU5-O, THR6-N, THR6-H, THR6-CA, THR6-HA, THR6-CB, THR6-HB, THR6-OG1, THR6-HG1, THR6-CG2, THR6-HG21, THR6-HG22, THR6-HG23, THR6-C, THR6-O, GLY7-N, GLY7-H, GLY7-CA, GLY7-HA3, GLY7-HA2, GLY7-C, GLY7-O, THR8-N, THR8-H, THR8-CA, 

/home/pnovelli/code/encoderops/.venv/lib/python3.11/site-packages/mlcolvar/utils/timelagged.py:142: UserWarning: Monitoring the progress for the search of time-lagged configurations with a progress_bar requires `tqdm`.
  warnings.warn(


Model init

Training

In [3]:
from lightning import Trainer
from mlcolvar.data import DictModule


trainer = Trainer(
    logger=False,
    enable_checkpointing=False,
    accelerator='cuda',
    max_epochs=10,
    enable_model_summary=True
)

datamodule = DictModule(dataset, batch_size=8)

Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
model_args = ModelArgs(latent_dim=512, encoder_lr=1e-3, linear_lr=1e-3, min_encoder_lr=1e-5, epochs=10, max_grad_norm= None, normalize_lin=True, regularization=1e-5)
model = EvolutionOperator(cutoff=dataset.metadata['cutoff'], atomic_numbers=dataset.metadata['z_table'],model_args= model_args)

In [5]:
trainer.fit(model, datamodule)

/home/pnovelli/code/encoderops/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


/home/pnovelli/code/encoderops/.venv/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:317: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.

  | Name    | Type            | Params | Mode 
----------------------------------------------------
0 | encoder | Sequential      | 18.6 K | train
1 | linear  | Linear          | 262 K  | train
2 | loss    | RegSpectralLoss | 0      | train
----------------------------------------------------
280 K     Trainable params
0         Non-trainable params
280 K     Total params
1.123     Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode
/home/pnovelli/code/encoderops/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_wor

Epoch 0:   4%|▎         | 2/54 [00:01<00:32,  1.62it/s]

/home/pnovelli/code/encoderops/.venv/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 1:  57%|█████▋    | 31/54 [00:04<00:03,  7.34it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined